In [258]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
df=pd.read_csv("./data/players.csv",low_memory=False)
df.describe()

,ATPRank,games,wins,losses,players_odds,indoor_wins,indoor_losses,outdoor_wins,outdoor_losses
count,1429.000000,1429.000000,1429.000000,1429.000000,896.000000,1429.000000,1429.000000,1429.000000,1429.000000
mean,303.211337,72.212736,36.106368,36.106368,3.461692,6.447166,6.447166,29.659202,29.659202
std,290.360729,153.371780,93.132552,66.013197,1.580138,17.592040,12.914480,77.252592,53.834708
min,0.000000,1.000000,0.000000,0.000000,1.135000,0.000000,0.000000,0.000000,0.000000
25%,129.000000,2.000000,0.000000,1.000000,2.658006,0.000000,0.000000,0.000000,1.000000
50%,222.000000,6.000000,2.000000,5.000000,3.147096,0.000000,1.000000,1.000000,4.000000
75%,378.000000,61.000000,22.000000,37.000000,3.749802,3.000000,6.000000,18.000000,32.000000
max,3065.000000,1223.000000,1017.000000,425.000000,18.230000,167.000000,94.000000,941.000000,346.000000


In [80]:
df.shape

elo 1500
gagne +32
perds -32


progression : 0.5
gagne 0.1 /anciennete

note cachée  = 1500
-1 par jour 
gangner/perdu + ancienneté match + rank atp adversaire

la note ajoute la temporalité




(1429, 13)